# New plotting and printing options in dream-tools 3.0.0
A few useful tools have been added or upgraded to help analyze time series and age profiles more easily.

Our convention is to import the *dreamtools* package under the alias *dt*.
Functions that we use a lot are imported directly, e.g. *plot* and *prt*. The plotly.renderer is set depending on the environment.


In [2]:
import dreamtools as dt
from dreamtools import plot, prt
import plotly.io as pio
pio.renderers.default = "jupyterlab"

We open a gdx file as a GamsPandasDatabase and set it as the "reference database".
The reference database should be our baseline scenario for any analysis.

In [4]:
dt.REFERENCE_DATABASE = dt.Gdx("Q:/MAKRO/Data/PreviousSolutions/previous_dynamic_calibration")

We read another gdx file and assign it to a short name for easy access.

In [6]:
b = dt.Gdx("Q:/MAKRO/Data/PreviousSolutions/previous_dynamic_calibration2")

Let's try plotting a few national income variables:

In [8]:
plot([b.vX, b.vC, b.vG, b.vI])

Note that only the aggregate elements are plotted despite each variable being multidimensional.

That is because *dreamtools* is setup with a dictionary defining which elements should be shown for a number of sets if we haven't specified anything.

In [10]:
dt.DEFAULT_SET_AGGREGATIONS

{'a_': ['tot'],
 'a': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
 'c_': ['cTot'],
 'c': ['cBil', 'cEne', 'cVar', 'cBol', 'cTje', 'cTur'],
 'x_': ['xTot'],
 'x': ['xEne', 'xVar', 'xSoe', 'xTje', 'xTur'],
 'g_': ['gTot'],
 'g': ['g'],
 's_': ['tot'],
 'sp': ['tje', 'fre', 'byg', 'soe', 'bol', 'lan', 'ene', 'udv'],
 's': ['tje', 'fre', 'byg', 'soe', 'bol', 'lan', 'ene', 'udv', 'off'],
 'r_': ['rTot'],
 'r': ['tje', 'fre', 'byg', 'soe', 'bol', 'lan', 'ene', 'udv', 'off'],
 'i_': ['iTot'],
 'i': ['im', 'ib', 'iL'],
 'k_': ['im', 'ib'],
 'k': ['im', 'ib'],
 'portf': ['Net'],
 'portf_': ['Net'],
 'scen_': ['GovSpending', 'IntRate', 'ForeignDemand']}

If we want other components we can simply be more specific (or supply a different *default_set_aggregations* dictionary):

In [12]:
plot(b.vC[b.c])

To compare variables with our baseline (the reference database) instead of writing

```
b.vX / baseline.vX - 1, b.vX / baseline.vX - 1 ...
```

we can use the operator "q" (as in Gekko):

In [14]:
plot([b.vX, b.vC, b.vG, b.vI], "q", 2015, 2025)

The *prt* function works identically, except instead of returning a plotly figure for plotting, it returns a string for printing.

We can set a global time period of interest using the *time* function.

3 operators are currently implemented:
- "s": show baseline side-by-side (shock, baseline)
- "q": relative multiplier (shock / baseline - 1)
- "m": absolute multiplyer (shock - baseline)



In [16]:
dt.time(2017, 2025)
prt(b.vX, "s")

,vX[xTot],baseline.vX[xTot]
2017,976.289020,976.289020
2018,946.749522,946.901117
2019,942.291394,942.681092
2020,963.373697,964.179449
2021,975.046998,976.381269
2022,982.077018,983.920739
2023,986.658988,988.927160
2024,989.739415,992.381343
2025,991.777057,994.816736


Here is an example with a few more useful arguments:

In [18]:
plot([b.qY, b.qK],
        operator="m",
        start_year=2016,
        end_year=2080,
        reference_database=None,
        default_set_aggregations={"s_": ["tje"], "s": ["tje"]},
        layout={"title": "Example graph", "yaxis": {"title": "Difference from baseline"}},  # See plotly documentation for other layout options
)

## Age profiles
2 functions have been added specifically to plot age profiles more easily:

In [20]:
dt.age_figure_2d(b.vC_a, years=[2017, 2090])

In [22]:
dt.age_figure_3d(b.vC_a, end_age=99, end_year=2099)


## Exporting
There are many ways to export the plots, but one easy way to store and share them is to export an html file:

In [24]:
figures = [
    plot(b.vC),
    plot(b.vG)
]
dt.figures_to_html(figures, "example.html")